# Imports

In [1]:
from modules.data import data_loader, data_splitter, data_reader
from modules.models import arima, lag_llama, autoregressor
from modules.evaluation import evaluate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from modules.experiment.experiment import run_experiment
from modules.data.tscv import create_tscv_dataset
from modules.experiment.tscv import get_tscv_results, mean_directional_accuracy, get_summary


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from itertools import islice
from sklearn.model_selection import TimeSeriesSplit

# Parameters

In [2]:
PREDICTION_LENGTH = 10 # currentlz only works for PREDICTION_LENGTH > 1
FREQUENCY = "daily" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama"] # currentlz works onlz for these two

# want to add
PREDICTION_HORIZON = 3 # can use anything as long as it complies with data length
#TRAIN_PERIOD = # context lenghts. Should take a look into this
#TRAIN_SIZE = 
FOLDS = 10# for TSCV
CONTEXT_LENGTH = 64



# Notes

 - autoregressor
 - mean directional accuracy
 - ask
 - for each time-series create a whole dataframe for TSCV
  - problem with this is with time horizon that is >1 
  - prediction horizon is only for that value in the future

# TO DO

 - create the TSCV experiment. It needs to create a table. Metrics for each fold
 - lit review
 - content for the presentation by monday!!
 - review data leakage risk
 - writing of the dissertation
 - autoregressor
 - MDA metric
 - frequency

 Writing
 - lit review
 - lit reading

# Loading the data

In [3]:
data = data_reader(type = TYPE_OF_DATA, frequency = FREQUENCY)
# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list
if len(data) > 1:
    data = data[0]

simple_data = data_loader.get_simle_data()

[*********************100%%**********************]  1 of 1 completed


# Prepare the data

In [4]:
# pass full data at maximum granularity and produce according to frequency parameter

In [6]:
tscv_data = create_tscv_dataset(data = simple_data, context_length=CONTEXT_LENGTH, n_folds=FOLDS, prediction_horizon=PREDICTION_HORIZON, max_folds=False)

# Models

In [7]:
# arima


In [8]:
# lag llama

# Run regular experiment

In [ ]:
results = run_experiment(data = data, prediction_length = PREDICTION_LENGTH)

# Evaluation

In [ ]:
results

# Run TSCV experiment

look up tzpes of TSCV

In [10]:
tscv_results, prediction = run_experiment(data=tscv_data, tscv=True)

y0 done  1
------------
y1 done  2
------------
y2 done  3
------------
y3 done  4
------------
y4 done  5
------------
y5 done  6
------------
y6 done  7
------------
y7 done  8
------------
y8 done  9
------------
y9 done  10
------------


In [11]:
tscv_results

{'r2': {'arima': -0.8036159605688358, 'llama': -4.690321882253741},
 'mse': {'arima': 9.014638013311012, 'llama': 28.440750730306206},
 'mae': {'arima': 2.712121007999187, 'llama': 4.178632465124013},
 'rmse': {'arima': 3.0024386776936867, 'llama': 5.332987036390226}}

# SKLearn TSCV

In [3]:
simple_data = data_loader.get_simle_data()

[*********************100%%**********************]  1 of 1 completed


In [4]:
# sklearn tscv object
tscv = TimeSeriesSplit(n_splits=FOLDS, test_size=PREDICTION_LENGTH)


In [5]:
# extracting the time-series data from original data
series = simple_data["y"]

- put metrics in dataframe. one for arima, one for lag llama  DONE
- each column metric, each row is a fold  DONE
- create wrapper function get_metrics(predicted, actual) returns pd.DataFrame of description above
- by end of day have a table   DONE
- agenda of the presentation by 12   DONE
- implement autoregressor forecast t1 = t0     DONE
- implement frequency experimentation          DONE
- start training date global parameter

In [ ]:
# initializing the list of models, metrics and emptz result dict
models=["arima", "llama"]
metrics=["r2", "mse", "mae", "rmse", "mda"]
results = {metric: {model: {f"fold_{i}": [] for i in range(FOLDS)} for model in models} for metric in metrics}

arima_results = pd.DataFrame(columns=metrics)
llama_results = pd.DataFrame(columns= metrics)
autoregressor_results = pd.DataFrame(columns=metrics)




i = 0
# iterating over all the folds
for train_index, test_index in tscv.split(series):
    # subsetting the original data according to train/test split
    train = simple_data.iloc[train_index]
    valid = list(simple_data.iloc[test_index]["y"])

    # for my testing purposes
    #print(train["y"])
    #print(train["y"].iloc[-1])


    # inputting data into the models
    arima_model = arima.get_autoarima(train)
    autoarima_predictions = arima.autoarima_predictions(arima_model, PREDICTION_LENGTH)
    lag_llama_predictions, tss = lag_llama.get_lam_llama_forecast(train, PREDICTION_LENGTH, context_length=CONTEXT_LENGTH)
    lag_llama_predictions = list(lag_llama_predictions[0].samples.mean(axis = 0))
    autoregressor_predictions = autoregressor.get_autoregressor_prediction(train, PREDICTION_LENGTH)

    """
    print(i)
    print("----------")
    print(len(lag_llama_predictions))
    print("--------------")
    print(len(valid))
    print("---------------")
    print(PREDICTION_LENGTH)
    print("------------------")
    print(r2_score(valid, lag_llama_predictions))
    """

    # for my own testing purposes
    """
    print(autoarima_predictions)
    print(lag_llama_predictions)
    print(valid)
    """

    # recording the metrics
    """
    results["r2"]["arima"][f"fold_{i}"].append(r2_score(valid, autoarima_predictions))
    results["mse"]["arima"][f"fold_{i}"].append(mean_squared_error(valid, autoarima_predictions))
    results["mae"]["arima"][f"fold_{i}"].append(mean_absolute_error(valid, autoarima_predictions))
    results["rmse"]["arima"][f"fold_{i}"].append(np.sqrt(mean_squared_error(valid, autoarima_predictions)))

    results["r2"]["llama"][f"fold_{i}"].append(r2_score(valid, lag_llama_predictions))
    results["mse"]["llama"][f"fold_{i}"].append(mean_squared_error(valid, lag_llama_predictions))
    results["mae"]["llama"][f"fold_{i}"].append(mean_absolute_error(valid, lag_llama_predictions))
    results["rmse"]["llama"][f"fold_{i}"].append(np.sqrt(mean_squared_error(valid, lag_llama_predictions)))
    """

    arima_metrics = [r2_score(valid, autoarima_predictions), 
               mean_squared_error(valid, autoarima_predictions), 
               mean_absolute_error(valid, autoarima_predictions),
               np.sqrt(mean_squared_error(valid, autoarima_predictions)),
               mean_directional_accuracy(valid, autoarima_predictions, train["y"].iloc[-1])]
    
    llama_metrics = [r2_score(valid, lag_llama_predictions), 
               mean_squared_error(valid, lag_llama_predictions), 
               mean_absolute_error(valid, lag_llama_predictions),
               np.sqrt(mean_squared_error(valid, lag_llama_predictions)),
               mean_directional_accuracy(valid, lag_llama_predictions, train["y"].iloc[-1])]
    
    autoregressor_metrics = [r2_score(valid, autoregressor_predictions), 
               mean_squared_error(valid, autoregressor_predictions), 
               mean_absolute_error(valid, autoregressor_predictions),
               np.sqrt(mean_squared_error(valid, autoregressor_predictions)),
               mean_directional_accuracy(valid, autoregressor_predictions, train["y"].iloc[-1])]

    arima_results = pd.concat([arima_results, pd.DataFrame([arima_metrics], columns=metrics)], ignore_index=True)
    llama_results = pd.concat([llama_results, pd.DataFrame([llama_metrics], columns=metrics)], ignore_index=True)
    autoregressor_results = pd.concat([autoregressor_results, pd.DataFrame([autoregressor_metrics], columns=metrics)], ignore_index=True)


    i += 1

In [18]:
arima_results

,r2,mse,mae,rmse,mda
0,-10.146053,33.415565,5.512960,5.780620,0.2
1,-1.069529,56.237796,5.979982,7.499186,0.8
2,-0.021127,5.389042,1.626925,2.321431,0.4
3,-0.130501,2.120663,1.296481,1.456249,0.6
4,0.009028,14.090569,3.312309,3.753741,0.5
5,-1.962910,94.272351,7.710001,9.709395,0.4
6,-2.799649,39.613602,5.353981,6.293934,0.1
7,-0.137856,7.045263,2.276720,2.654291,0.9
8,-1.762008,200.092281,12.148269,14.145398,0.4
9,-0.884450,68.486433,6.201827,8.275653,0.1


In [19]:
arima_summary = pd.DataFrame({
    'r2': [arima_results['r2'].mean(), arima_results['r2'].median(), arima_results['r2'].std()],
    'mse': [arima_results['mse'].mean(), arima_results['mse'].median(), arima_results['mse'].std()],
    'mae': [arima_results['mae'].mean(), arima_results['mae'].median(), arima_results['mae'].std()],
    'rmse': [arima_results['rmse'].mean(), arima_results['rmse'].median(), arima_results['rmse'].std()],
    'mda': [arima_results['mda'].mean(), arima_results['mda'].median(), arima_results['mda'].std()]
}, index=['mean', 'median', 'std'])

In [20]:
arima_summary

,r2,mse,mae,rmse,mda
mean,-1.890506,52.076356,5.141945,6.188990,0.440000
median,-0.976990,36.514584,5.433470,6.037277,0.400000
std,3.054336,60.208310,3.272644,3.911914,0.271621


In [21]:
llama_results

,r2,mse,mae,rmse,mda
0,-11.283636,36.826008,5.202384,6.068444,0.6
1,-6.114432,193.328999,12.492389,13.904280,0.3
2,-27.709435,151.515247,12.034618,12.309153,0.5
3,-117.051050,221.447375,14.750225,14.881108,0.5
4,-10.914719,169.414716,12.395660,13.015941,0.5
5,-2.360813,106.932600,9.050172,10.340822,0.6
6,-2.001025,31.287475,4.966641,5.593521,0.6
7,-10.787917,72.987215,7.751465,8.543256,0.4
8,-7.082025,585.498344,22.845116,24.197073,0.5
9,-26.135169,986.171396,30.635747,31.403366,0.4


In [22]:
llama_summary = pd.DataFrame({
    'r2': [llama_results['r2'].mean(), llama_results['r2'].median(), llama_results['r2'].std()],
    'mse': [llama_results['mse'].mean(), llama_results['mse'].median(), llama_results['mse'].std()],
    'mae': [llama_results['mae'].mean(), llama_results['mae'].median(), llama_results['mae'].std()],
    'rmse': [llama_results['rmse'].mean(), llama_results['rmse'].median(), llama_results['rmse'].std()],
    'mda': [llama_results['mda'].mean(), llama_results['mda'].median(), llama_results['mda'].std()]
}, index=['mean', 'median', 'std'])

In [23]:
llama_summary

,r2,mse,mae,rmse,mda
mean,-22.144022,255.540937,13.212442,14.025696,0.490000
median,-10.851318,160.464981,12.215139,12.662547,0.500000
std,34.493446,301.754252,8.031871,8.084332,0.099443


In [25]:
autoregressor_results

,r2,mse,mae,rmse,mda
0,-9.748225,32.222889,5.406007,5.676521,0.0
1,-1.338956,63.559268,6.378003,7.972407,0.0
2,-0.056281,5.574567,1.645000,2.361052,0.0
3,-0.427978,2.678689,1.514003,1.636670,0.0
4,-0.004325,14.280447,3.302000,3.778948,0.0
5,-1.668451,84.903390,7.286008,9.214304,0.0
6,-2.683164,38.399180,5.289006,6.196707,0.0
7,-0.095014,6.779998,2.207004,2.603843,0.0
8,-1.774466,200.994832,12.166003,14.177265,0.0
9,-0.709524,62.129100,5.946002,7.882201,0.0


In [24]:
llama_summary - arima_summary

,r2,mse,mae,rmse,mda
mean,-20.253517,203.464581,8.070496,7.836706,0.050000
median,-9.874328,123.950398,6.781669,6.625270,0.100000
std,31.439111,241.545942,4.759227,4.172418,-0.172178


In [4]:
r = get_tscv_results(simple_data, PREDICTION_HORIZON, CONTEXT_LENGTH, FOLDS)

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\experiment\tscv.py:130: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  arima_results = pd.concat([arima_results, pd.DataFrame([arima_metrics], columns=metrics)], ignore_index=True)
c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\experiment\tscv.py:131: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  llama_results = pd.concat([llama_res

In [5]:
r[1]

,r2,mse,mae,rmse,mda
0,-16.258888,36.976490,5.325571,6.080830,0.666667
1,-36.662651,24.117652,4.749936,4.910973,0.333333
2,-190.402401,123.313290,10.915252,11.104652,0.000000
3,-32.929511,82.451167,9.035596,9.080263,1.000000
4,-77.796570,758.931083,27.164961,27.548704,0.000000
5,-292.419610,859.938481,29.238678,29.324708,0.333333
6,-605.704627,511.662005,22.588960,22.619947,0.666667
7,-152.117397,739.994888,26.966544,27.202847,0.333333
8,-52.965865,857.997695,28.725477,29.291598,0.333333
9,-231.179545,1382.088060,36.906473,37.176445,0.000000


In [8]:
get_summary(r[1])

,r2,mse,mae,rmse,mda
mean,-168.843706,537.747081,20.161745,20.434096,0.366667
median,-114.956983,625.828446,24.777752,24.911397,0.333333
std,180.028594,459.881932,11.567582,11.556373,0.331476
